In [1]:
import pandas as pd
df = pd.read_csv("train.csv")

In [2]:
df = df.drop(["Place", "ID", "date", "status", "job_title", "advice_to_mgmt", "location"], axis=1)

In [3]:
df.head()

summary  \
0                           Best Company to work for   
1  Moving at the speed of light, burn out is inev...   
2  Great balance between big-company security and...   
3  The best place I've worked and also the most d...   
4                           Execellent for engineers   

                                           positives  \
0                      People are smart and friendly   
1  1) Food, food, food. 15+ cafes on main campus ...   
2  * If you're a software engineer, you're among ...   
3  You can't find a more well-regarded company th...   
4             Impact driven. Best tech in the world.   

                                           negatives  score_1  score_2  \
0                 Bureaucracy is slowing things down      4.0      5.0   
1  1) Work/life balance. What balance? All those ...      2.0      3.0   
2  * It *is* becoming larger, and with it comes g...      5.0      4.0   
3  I live in SF so the commute can take between 1...      2.0      5.0   
4  Size matters. Engineers are a bit disconnected...      5.0      5.0   

   score_3  score_4  score_5  score_6  overall  
0      5.0      4.0      5.0        0      5.0  
1      3.0      5.0      3.0     2094      5.0  
2      5.0      5.0      4.0      949      5.0  
3      5.0      4.0      5.0      498      4.0  
4      5.0      5.0      5.0        0      4.0

In [ ]:
df["score_5"].value_counts()

In [4]:
df["score_1"] = df["score_1"].fillna(3.0)
df["score_2"] = df["score_2"].fillna(5.0)
df["score_3"] = df["score_3"].fillna(3.0)
df["score_4"] = df["score_4"].fillna(4.0)
df["score_5"] = df["score_5"].fillna(5.0)
df["summary"] = df["summary"].fillna("Great Place to work")

#### Text Prepocessing

In [5]:
# Text processing

from nltk.corpus import stopwords
sw = stopwords.words("english")

import re
from nltk.stem.snowball import SnowballStemmer
import string

def clean_text(text):

    ## Remove puncuation
    text = text.translate(string.punctuation)

    ## Convert words to lower case and split them
    text = text.lower().split()

    ## Remove stop words
    text = [w for w in text if not w in sw and len(w) >= 3]

    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    txt = text.split()
    text = [i for i in txt if i.isalpha()]
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [6]:
df["summary"] = df["summary"].apply(clean_text)
df["positives"] = df["positives"].apply(clean_text)
df["negatives"] = df["negatives"].apply(clean_text)

In [11]:
tf_train_data = pd.concat([df['positives'], df['negatives']])

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df=3, max_features=1000,
strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)

In [13]:
tfv.fit(tf_train_data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words=None, strip_accents='unicode', sublinear_tf=1,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=1,
        vocabulary=None)

In [48]:
desc1 = tfv.fit_transform(list(df["summary"]))
desc2 = tfv.transform(list(df["positives"]))
desc3 = tfv.transform(list(df["negatives"]))

# truncate the matrix/ array to 20 column
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=40)
desc1 = svd.fit_transform(desc1)
desc2 = svd.fit_transform(desc2)
desc3 = svd.fit_transform(desc3)

desc1 = pd.DataFrame(desc1, columns=['nlp_1_{}'.format(i) for i in range(40)])
desc2 = pd.DataFrame(desc2, columns=['nlp_2_{}'.format(i) for i in range(40)])
desc3 = pd.DataFrame(desc3, columns=['nlp_3_{}'.format(i) for i in range(40)])

final_df = pd.concat([df,desc2,desc3],axis=1, sort=False)

In [49]:
Y = final_df["overall"]
final_df = final_df.drop(["summary","positives", "negatives", "overall"], axis =1)

In [50]:
final_df.head()

score_1  score_2  score_3  score_4  score_5  score_6   nlp_2_0   nlp_2_1  \
0      4.0      5.0      5.0      4.0      5.0        0  0.256700 -0.315508   
1      2.0      3.0      3.0      5.0      3.0     2094  0.246814 -0.025985   
2      5.0      4.0      5.0      5.0      4.0      949  0.309547 -0.144818   
3      2.0      5.0      5.0      4.0      5.0      498  0.305647 -0.170771   
4      5.0      5.0      5.0      5.0      5.0        0  0.055810 -0.050554   

    nlp_2_2   nlp_2_3    ...     nlp_3_30  nlp_3_31  nlp_3_32  nlp_3_33  \
0 -0.245085 -0.104157    ...    -0.153254 -0.037407 -0.088008 -0.007320   
1 -0.028924  0.026376    ...     0.049169 -0.027655 -0.034571  0.003213   
2  0.124553  0.018168    ...    -0.003165 -0.126918  0.045242 -0.068055   
3 -0.067501  0.074886    ...    -0.103069 -0.027513  0.053910  0.046382   
4 -0.014839  0.001218    ...     0.089727  0.039118  0.046491  0.043190   

   nlp_3_34  nlp_3_35  nlp_3_36  nlp_3_37  nlp_3_38  nlp_3_39  
0 -0.194742  0.261765  0.251933 -0.125418  0.013273 -0.032208  
1 -0.000993 -0.046402  0.034600  0.027009  0.009137  0.010900  
2  0.039860  0.137792 -0.012873 -0.059764  0.076429  0.023346  
3 -0.006424  0.094134 -0.037048 -0.063302  0.024819 -0.069581  
4 -0.099383 -0.012495 -0.043670  0.110545  0.024121  0.078854  

[5 rows x 86 columns]

In [51]:
#final_df = final_df.drop(["score_1", "score_2", "score_3","score_4", "score_5", "score_6"], axis =1)

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(final_df, Y, test_size=0.25, random_state=42)

#### Logistic regression

In [53]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', C=1,multi_class='multinomial').fit(X_train, Y_train)
Y_pred = clf.predict(X_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
from sklearn.metrics import accuracy_score
acc_3 = accuracy_score(Y_test, Y_pred)

In [54]:
acc_3

0.40400843881856541

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0).fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

acc_DT = accuracy_score(Y_test, Y_pred)

In [ ]:
acc_DT

#### XGBoost

In [55]:
from xgboost import XGBClassifier


xgb = XGBClassifier(learning_rate=0.1,n_estimators=300, booster='dart')
xgb.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [56]:
Y_test = Y_test.tolist()
Y_pred = xgb.predict(X_test)
Y_pred = Y_pred.tolist()

In [57]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
from sklearn.metrics import accuracy_score
acc_3 = accuracy_score(Y_test, Y_pred)

In [58]:
acc_3

0.41323839662447259

In [ ]:
acc_3

In [ ]:
Y_test = Y_test.tolist()

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

rf.fit(X_train, Y_train)

Y_pred = rf.predict(X_test)


Y_pred = Y_pred.tolist()

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
from sklearn.metrics import accuracy_score
acc_3 = accuracy_score(Y_test, Y_pred)

In [ ]:
acc_3

#### cat boost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model = CatBoostClassifier(
    iterations=200,
    learning_rate=1,
    loss_function='MultiClass',
    depth = 2
    # loss_function='CrossEntropy'
)
model.fit(
    X_train, Y_train,
    #cat_features=cat_features,
    eval_set=(X_test, Y_test),
    verbose=False
    
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

In [ ]:
Y_pred = model.predict(data=X_test)

In [ ]:
Y_pred = Y_pred.tolist()

In [ ]:
Y_pred = [int(Y_pred[i][0]) for i in range(len(Y_pred))]

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)
from sklearn.metrics import accuracy_score
acc_3 = accuracy_score(Y_test, Y_pred)

In [ ]:
acc_3

#### Test_File

In [60]:
test = pd.read_csv("test.csv")

In [61]:
test = test.drop(["Place", "ID", "date", "status", "job_title", "advice_to_mgmt", "location"], axis=1)

In [62]:
test["score_1"] = test["score_1"].fillna(3.0)
test["score_2"] = test["score_2"].fillna(5.0)
test["score_3"] = test["score_3"].fillna(3.0)
test["score_4"] = test["score_4"].fillna(4.0)
test["score_5"] = test["score_5"].fillna(5.0)
test["summary"] = test["summary"].fillna("Great Place to work")

In [63]:
test["negatives"]= test["negatives"].fillna("None that I can think of.")

In [64]:
test["summary"] = test["summary"].apply(clean_text)
test["positives"] = test["positives"].apply(clean_text)
test["negatives"] = test["negatives"].apply(clean_text)

In [65]:
#desc1 = tfv.fit_transform(list(test["summary"]))
desc2 = tfv.transform(list(test["positives"]))
desc3 = tfv.transform(list(test["negatives"]))

#desc1 = svd.fit_transform(desc1)
desc2 = svd.fit_transform(desc2)
desc3 = svd.fit_transform(desc3)

#desc1 = pd.DataFrame(desc1, columns=['nlp_1_{}'.format(i) for i in range(20)])
desc2 = pd.DataFrame(desc2, columns=['nlp_2_{}'.format(i) for i in range(40)])
desc3 = pd.DataFrame(desc3, columns=['nlp_3_{}'.format(i) for i in range(40)])

final_df_test = pd.concat([test,desc2,desc3],axis=1, sort=False)

In [66]:
final_df_test = final_df_test.drop(["summary", "positives", "negatives"], axis=1)

In [67]:
final_df_test.head()

score_1  score_2  score_3  score_4  score_5  score_6   nlp_2_0   nlp_2_1  \
0        5        5        5        5        5       49  0.390834 -0.195544   
1        4        4        4        5        4        1  0.205144 -0.174499   
2        5        4        4        5        4        0  0.348355 -0.159631   
3        5        5        5        5        5        0  0.435898  0.026453   
4        5        5        5        5        5        0  0.147673 -0.014842   

    nlp_2_2   nlp_2_3    ...     nlp_3_30  nlp_3_31  nlp_3_32  nlp_3_33  \
0  0.067739 -0.085525    ...    -0.044793 -0.025591  0.055054  0.041824   
1 -0.115875 -0.031276    ...    -0.099369 -0.129594  0.013185 -0.029079   
2  0.653501  0.143479    ...    -0.042463 -0.106183  0.116946 -0.194297   
3 -0.090527  0.181535    ...     0.038608 -0.041113 -0.030246  0.049204   
4 -0.002035 -0.056914    ...    -0.031555 -0.043925 -0.065572 -0.058551   

   nlp_3_34  nlp_3_35  nlp_3_36  nlp_3_37  nlp_3_38  nlp_3_39  
0 -0.035216 -0.110948 -0.035874 -0.003658  0.012142  0.055254  
1 -0.041803  0.143983 -0.027241 -0.026675  0.041362 -0.044608  
2  0.015419  0.071907  0.042702 -0.129840 -0.044489  0.055303  
3  0.021807  0.051377  0.004602  0.021222 -0.035577  0.000709  
4 -0.025958 -0.036685 -0.012644 -0.019774  0.040902 -0.044799  

[5 rows x 86 columns]

In [74]:
Pred = clf.predict(final_df_test)

In [75]:
Pred_f = Pred.tolist()

In [76]:
# Used for Catboost
#Pred_f = [int(Pred_f[i][0]) for i in range(len(Pred_f))]

In [77]:
test_df = pd.read_csv("test.csv")

In [78]:
submit = pd.DataFrame({"overall":Pred_f})
submit.insert(loc=0, column='ID', value=test_df["ID"])

In [79]:
submit.to_csv('submission_final.csv',index=False)

In [ ]:
submit